In [22]:
import json
import statistics

In [35]:
lines = open('brucewick_60.lo2g').readlines()

In [36]:
js = []
for lno,line in enumerate(lines):
    try:
        js.append(json.loads(line))
    except Exception as e:
        print(f"{lno}) {e}")
        continue

In [37]:
js

[{'time': '2022-01-17T11:32:27.427812-08:00',
  'epoch_secs': 1642447947,
  'epoch_ms': 427,
  'name': 'timings',
  'level': 'info',
  'process': 19300,
  'thread': 19300,
  'message': {'type': 'stopwatch_start',
   'name': 'sim',
   'id': 7,
   'id_hierarchy': []}},
 {'time': '2022-01-17T11:32:27.427843-08:00',
  'epoch_secs': 1642447947,
  'epoch_ms': 427,
  'name': 'timings',
  'level': 'info',
  'process': 19300,
  'thread': 19300,
  'message': {'type': 'stopwatch_start',
   'name': 'frame_0001',
   'id': 8,
   'id_hierarchy': ['7']}},
 {'time': '2022-01-17T11:32:27.427846-08:00',
  'epoch_secs': 1642447947,
  'epoch_ms': 427,
  'name': 'timings',
  'level': 'info',
  'process': 19300,
  'thread': 19300,
  'message': {'type': 'stopwatch_start',
   'name': 'frame_serialization',
   'id': 9,
   'id_hierarchy': ['7', '8']}},
 {'time': '2022-01-17T11:32:27.437271-08:00',
  'epoch_secs': 1642447947,
  'epoch_ms': 437,
  'name': 'timings',
  'level': 'info',
  'process': 19300,
  'thread

In [38]:
timings = {}
step_ids = {}
id_parents = {}
id_names = {}
for j in js:
    if "name" in j and j["name"] == "timings":
        info = j["message"]
        name = info["name"]
        myid = info["id"]
        id_names[myid] = name
        if name.startswith('frame_'):
            name,index = name.split('_')
            try:
                index = int(index)
                step_ids[myid] = index
                
                #print(info["name"])
                #print(info)
                if index not in timings:
                    timings[index] = {}
            except Exception as e:
                pass

        if info["type"] == "stopwatch_start":
            myset = set()
            for id in info["id_hierarchy"]:
                try:
                    myset.add(step_ids[int(id)])
                except Exception as e:
                    pass
            id_parents[myid] = myset
            continue
        elif info["type"] != "stopwatch_end":
            continue
        duration = info["duration"]
        if myid not in id_parents:
            continue
        for pid in id_parents[myid]:
            if pid not in timings:
                continue
            mytimings = timings[pid]
            if name not in mytimings:
                #print(info)
                dat = duration
            else:
                dat = mytimings[name] + duration
            mytimings[name] = dat
            timings[pid] = mytimings
print(timings)
#print(id_names)

{1: {'frame': 9.422597, 'Sim::update_polynomial_velocity': 14.4515, 'Sim::update_particle_cell_cache': 9476.474710999999, 'Sim::initialize_particles': 7244.995738, 'Sim::save_frame': 1352.360764, 'Sim::update_fluxes_using_particles': 23157.91824, 'Sim::update_moments_from_particles': 24.561048, 'Sim::update_particles_to_samples': 23182.539078, 'Sim::update_divergence_from_velocity': 30.781562, 'CGSolve_ramp': 3646.647312, 'Sim::update_pressure_from_divergence': 3648.508293, 'Sim::update_pressure_gradient_from_pressure': 14.928721, 'Sim::pressure_projection': 3694.854227, 'Sim::advect_particles_with_field': 1125.855339, 'Sim::advect_particles': 6077.619459, 'Sim::advect': 6077.631229, 'substep_0000': 6083.598594, 'advection': 6083.943524, 'Sim::step': 36676.20483}, 2: {'frame': 10.140948, 'Sim::save_frame': 3364.704277, 'Sim::update_particle_cell_cache': 7077.091036000001, 'Sim::update_fluxes_using_particles': 23181.838447, 'Sim::update_moments_from_particles': 24.540008, 'Sim::update_p

In [39]:
events = {}
counts = {}
for eid,v in timings.items():
    for event,v in v.items():
        if event not in events:
            events[event] = {}
        events[event][eid] = v
keys = ["Sim::step",
"Sim::pressure_projection",
"Sim::update_particle_cell_cache",
"Sim::update_particles_to_samples",
"advection",
"Sim::save_frame"]

valid_ids = [set(events[key].keys()) for key in keys]

current = valid_ids[0]
for ids in valid_ids[1:]:
    current = current.intersection(ids)
print(current)
filtered_events = {}
averages = {}
for key in keys:
    event = events[key]
    filtered_events[key] = {id:event[id] for id in current}
    averages[key] = statistics.mean(filtered_events[key].values())

#print(filtered_events)
    
print(averages)
 
"""for name,event in events.items():
    print(len(event))
for v in timings.values():
    for k,v in v.items():
        events[k] += v
        counts[k] += 1
        
"""
#averages = {name: (pr[0] / pr[1]) for name,pr in timings.items()}


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

'for name,event in events.items():\n    print(len(event))\nfor v in timings.values():\n    for k,v in v.items():\n        events[k] += v\n        counts[k] += 1\n        \n'

In [40]:
for name,time in averages.items():
    print(f"{name}: {time}")

step = averages[keys[0]]
pp = averages[keys[1]]
cache = averages[keys[2]]
p2m = averages[keys[3]]
m2p = averages[keys[4]]
save = averages[keys[5]]
print(m2p+p2m+cache+pp+save,step)

print(f"{step:.3f} & 0 & 0 & {cache:.3f} & {p2m:.3f} & {pp:.3f} & {m2p:.3f}")

Sim::step: 75648.52349444578
Sim::pressure_projection: 37016.5502455261
Sim::update_particle_cell_cache: 9875.63541731727
Sim::update_particles_to_samples: 23144.15674574297
advection: 9592.534897032128
Sim::save_frame: 3552.3759643293174
83181.25326994779 75648.52349444578
75648.523 & 0 & 0 & 9875.635 & 23144.157 & 37016.550 & 9592.535


In [54]:


def process(filename, keys):
    lines = open(filename).readlines()
    js = []
    for line in lines:
        try:
            js.append(json.loads(line))
        except Exception as e:
            print(f"Fail {e}")
            continue
    #print(f"Parsed js {len(js)}")
    timings = {}
    step_ids = {}
    id_parents = {}
    id_names = {}
    for j in js:
        if "name" in j and j["name"] == "timings":
            info = j["message"]
            name = info["name"]
            myid = info["id"]
            id_names[myid] = name
            if name.startswith('frame_'):
                name,index = name.split('_')
                try:
                    index = int(index)
                    step_ids[myid] = index

                    #print(info["name"])
                    #print(info)
                    if myid not in timings:
                        timings[myid] = {}
                except Exception as e:
                    pass

            if info["type"] == "stopwatch_start":
                id_parents[myid] = set((int(id) for id in info["id_hierarchy"]))
                continue
            elif info["type"] != "stopwatch_end":
                continue
            duration = info["duration"]
            if myid not in id_parents:
                continue
            for pid in id_parents[myid]:
                if pid not in timings:
                    continue
                mytimings = timings[pid]
                if name not in mytimings:
                    #print(info)
                    dat = duration
                else:
                    dat = mytimings[name] + duration
                mytimings[name] = dat
                timings[pid] = mytimings
    events = {}
    counts = {}
    for eid,v in timings.items():
        for event,v in v.items():

            if event not in events:
                events[event] = {}
            events[event][eid] = v
    print(filename)
    print(events)
    valid_ids = [set(events[key].keys()) for key in keys]
    current = valid_ids[0]
    for ids in valid_ids[1:]:
        current = current.intersection(ids)
    filtered_events = {}
    averages = {}

    for key in keys:
        event = events[key]
        filtered_events[key] = {id:event[id] for id in current}
        averages[key] = statistics.mean(filtered_events[key].values())
    
    return averages


def process_steps(filename):
    keys = ["Sim::step",
    "Sim::pressure_projection",
    "Sim::update_particle_cell_cache",
    "Sim::update_particles_to_samples",
    "advection",
    "Sim::save_frame"]
    averages = process(filename, keys)
    step = averages[keys[0]] / 1000
    pp = averages[keys[1]] / 1000
    cache = averages[keys[2]] / 1000
    p2m = averages[keys[3]] / 1000
    m2p = averages[keys[4]] / 1000
    save = averages[keys[5]] / 1000
    #print(m2p+p2m+cache+pp+save,step)

    print(f"{step:.3f} & {cache:.3f} & {p2m:.3f} & {pp:.3f} & {m2p:.3f}\\\\")





In [50]:
process_steps("torus_50.log")

27.264 & 4.094 & 13.456 & 6.879 & 3.536\\


In [39]:
process_steps("logs_50.log")

Parsed js 10550
227.909 & 4.242 & 15.179 & 205.239 & 4.149\\


In [68]:
process("bunny_50.log")

Valid ids: 92
Sim::step: 487145.574932
Sim::pressure_projection: 457573.07929925
Sim::update_particle_cell_cache: 5452.942610336957
Sim::update_particles_to_samples: 20731.670212402172
advection: 5485.741682695652
Sim::save_frame: 1982.0136658586957
487.146 & 5.453 & 20.732 & 457.573 & 5.486\\


In [69]:
process("brucewick_60.log")

Valid ids: 249
Sim::step: 75648.52349444578
Sim::pressure_projection: 37016.5502455261
Sim::update_particle_cell_cache: 9875.63541731727
Sim::update_particles_to_samples: 23144.15674574297
advection: 9592.534897032128
Sim::save_frame: 3552.3759643293174
75.649 & 9.876 & 23.144 & 37.017 & 9.593\\


In [70]:
process("hilbert_50.log")

Valid ids: 87
Sim::step: 1218726.419969069
Sim::pressure_projection: 1190461.8843732644
Sim::update_particle_cell_cache: 4496.234168114943
Sim::update_particles_to_samples: 19631.772631264368
advection: 5094.79966262069
Sim::save_frame: 2044.4282213793103
1218.726 & 4.496 & 19.632 & 1190.462 & 5.095\\


In [71]:
process("cow_50.log")

Valid ids: 249
Sim::step: 56772.96058646185
Sim::pressure_projection: 33789.052207040164
Sim::update_particle_cell_cache: 5996.235109678715
Sim::update_particles_to_samples: 13830.994471148595
advection: 5749.120720305221
Sim::save_frame: 2041.2201708955824
56.773 & 5.996 & 13.831 & 33.789 & 5.749\\


In [72]:
process("dragon_50.log")

FileNotFoundError: [Errno 2] No such file or directory: 'dragon_50.log'

In [73]:
process("plane_50.log")

Valid ids: 249
Sim::step: 24465.633932345383
Sim::pressure_projection: 4423.187189369478
Sim::update_particle_cell_cache: 4081.8650701485944
Sim::update_particles_to_samples: 13157.832030955824
advection: 3450.937967056225
Sim::save_frame: 2066.6253423333333
24.466 & 4.082 & 13.158 & 4.423 & 3.451\\


In [74]:
process("face_50.log")

Valid ids: 249
Sim::step: 172454.8765276747
Sim::pressure_projection: 143882.47856846184
Sim::update_particle_cell_cache: 6470.261455634538
Sim::update_particles_to_samples: 18344.837863180725
advection: 6819.829690052209
Sim::save_frame: 2036.5297482008032
172.455 & 6.470 & 18.345 & 143.882 & 6.820\\


In [75]:
process("gummy-jack_50.log")

Valid ids: 193
Sim::step: 253786.07188279793
Sim::pressure_projection: 225391.753934171
Sim::update_particle_cell_cache: 7643.618595321243
Sim::update_particles_to_samples: 16550.093468756477
advection: 8471.195304046632
Sim::save_frame: 2005.6970024663212
253.786 & 7.644 & 16.550 & 225.392 & 8.471\\


In [76]:
process("yeahright_50.log")

Valid ids: 249
Sim::step: 82401.33927806828
Sim::pressure_projection: 55309.912081481925
Sim::update_particle_cell_cache: 5901.507998441767
Sim::update_particles_to_samples: 17739.814917064257
advection: 5944.752680634538
Sim::save_frame: 2037.7628580361445
82.401 & 5.902 & 17.740 & 55.310 & 5.945\\


In [42]:
process_steps("monkey_50.log")

Parsed js 10550
67.345 & 4.201 & 13.455 & 46.637 & 3.809\\


In [78]:
process("vent_50.log")

Valid ids: 249
Sim::step: 31228.13854992771
Sim::pressure_projection: 8706.70425071486
Sim::update_particle_cell_cache: 5956.338987477911
Sim::update_particles_to_samples: 13213.923371064257
advection: 5921.470788321285
Sim::save_frame: 2028.9727710441766
31.228 & 5.956 & 13.214 & 8.707 & 5.921\\


In [79]:
process("crwth_32.log")

Valid ids: 100
Sim::step: 794121.21220528
Sim::pressure_projection: 771729.24826564
Sim::update_particle_cell_cache: 4222.13828671
Sim::update_particles_to_samples: 15248.65154366
advection: 3724.00469009
Sim::save_frame: 2050.35660502
794.121 & 4.222 & 15.249 & 771.729 & 3.724\\


In [57]:
process("dragon_50.log")

TypeError: process() missing 1 required positional argument: 'keys'

In [65]:
def process2(filename, keys):
    lines = open(filename).readlines()
    js = []
    for line in lines:
        try:
            js.append(json.loads(line))
        except Exception as e:
            print(f"Fail {e}")
            continue
    #print(f"Parsed js {len(js)}")
    timings = {}
    step_ids = {}
    id_parents = {}
    id_names = {}
    for j in js:
        if "name" in j and j["name"] == "timings":
            info = j["message"]
            name = info["name"]
            myid = info["id"]
            if info["type"] != "stopwatch_end":
                continue
            duration = info["duration"]
            timings[name] = duration
            
    return timings
def process_precompute(filename):
    try:
        print(filename)
        keys = [
            "operators",
            "MandolineGeometry"
        ]
        values = process2(filename,keys)
        op = values[keys[0]]/1000
        geo = values[keys[1]] / 1000
        print(f"{geo:.3f} & {op:.3f} &")
    except:
        return
    

In [66]:
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/brucewick_60.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/bunny_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/cow_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/crwth_32.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/dragon_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/face_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/gummy-jack_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/hilbert_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_10.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_20.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_30.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_40.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_60.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/monkey_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/plane_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/sphere_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/torus_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/vent_50.log')
process_precompute('/home/mtao/git/jupyter-sandbox/sim_timings/init/yeahright_50.log')

/home/mtao/git/jupyter-sandbox/sim_timings/init/brucewick_60.log
2.834 & 30.757 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/bunny_50.log
4.988 & 31.219 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/cow_50.log
1.525 & 19.628 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/crwth_32.log
/home/mtao/git/jupyter-sandbox/sim_timings/init/dragon_50.log
5.835 & 49.696 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/face_50.log
3.595 & 32.775 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/gummy-jack_50.log
3.740 & 24.132 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/hilbert_50.log
6.233 & 30.704 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_10.log
0.176 & 0.678 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_20.log
0.505 & 2.118 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_30.log
1.078 & 5.145 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_40.log
2.311 & 10.827 &
/home/mtao/git/jupyter-sandbox/sim_timings/init/logs_50.log
3.744 & 20.024 &
/h